In [19]:
import os
import re
# import spacy
from langchain_community.document_loaders import UnstructuredMarkdownLoader, PythonLoader, DirectoryLoader
from langchain_text_splitters import MarkdownHeaderTextSplitter, RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.schema import Document
import ast

from vectorisation import FAISSVectorStore

# Инициализация spaCy для NLP-анализа
# nlp = spacy.load("en_core_web_sm")

# 1. Автоматизация связывания: Поиск упоминаний .py файлов в .md файлах
# def extract_py_references(md_content):
#     """Извлекает имена .py файлов из Markdown контента с помощью регулярных выражений и spaCy."""
#     py_files = set()
#     # Регулярное выражение для поиска имен файлов (например, document_loader.py)
#     pattern = r'[\w_]+\.py'
#     matches = re.findall(pattern, md_content)
#     py_files.update(matches)
    
#     # Использование spaCy для дополнительного анализа (поиск упоминаний файлов в контексте)
#     doc = nlp(md_content)
#     for ent in doc.ents:
#         if ent.text.endswith('.py'):
#             py_files.add(ent.text)
    
#     return list(py_files)

# def build_file_relations(md_files, directory):
#     """Создает словарь связей между .md и .py файлами."""
#     file_relations = {}
#     for md_file in md_files:
#         with open(md_file, 'r', encoding='utf-8') as f:
#             content = f.read()
#         py_references = extract_py_references(content)
#         # Проверяем, существуют ли упомянутые .py файлы в директории
#         existing_py_files = [
#             os.path.join(root, f) for root, _, files in os.walk(directory) for f in files
#             if f in py_references and f.endswith('.py')
#         ]
#         file_relations[os.path.basename(md_file)] = existing_py_files
#     return file_relations

# 2. Загрузка и чанкирование .md файлов
def load_md_files(directory):
    """Загружает и чанкирует .md файлы с учетом структуры заголовков."""
    loader = DirectoryLoader(
        directory,
        glob="**/*.md",
        loader_cls=UnstructuredMarkdownLoader,
        loader_kwargs={"mode": "elements", "strategy": "fast"},
        silent_errors=True  # Обработка ошибок: пропуск проблемных файлов
    )
    docs = loader.load()
    
    headers_to_split_on = [("#", "Header 1"), ("##", "Header 2"), ("###", "Header 3")]
    splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,  # Оптимизация: размер чанка для больших файлов
        chunk_overlap=200  # Оптимизация: перекрытие для сохранения контекста
    )
    
    md_chunks = []
    for doc in docs:
        if doc.metadata.get("category") == "NarrativeText":
            # Чанкирование по заголовкам
            chunks = splitter.split_text(doc.page_content)
            for chunk in chunks:
                chunk.metadata.update({"source_md": doc.metadata["source"]})
                md_chunks.append(chunk)
        else:
            # Для других элементов (например, код) используем текстовый сплиттер
            chunks = text_splitter.split_text(doc.page_content)
            for chunk in chunks:
                md_chunks.append(Document(page_content=chunk, metadata={"source_md": doc.metadata["source"]}))
    
    return md_chunks

# 3. Загрузка и чанкирование .py файлов
def load_py_files(directory):
    """Загружает и чанкирует .py файлы с учетом структуры кода."""
    loader = DirectoryLoader(
        directory,
        glob="**/*.py",
        loader_cls=PythonLoader,
        silent_errors=True  # Обработка ошибок: пропуск проблемных файлов
    )
    docs = loader.load()
    
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,  # Оптимизация: размер чанка для больших файлов
        chunk_overlap=200  # Оптимизация: перекрытие для сохранения контекста
    )
    py_chunks = []
    
    for doc in docs:
        code = doc.page_content
        try:
            # Парсинг структуры Python с помощью ast
            tree = ast.parse(code)
            for node in ast.walk(tree):
                if isinstance(node, (ast.FunctionDef, ast.ClassDef)):
                    start_line = node.lineno
                    end_line = node.end_lineno
                    chunk = "\n".join(code.splitlines()[start_line-1:end_line])
                    py_chunks.append(Document(
                        page_content=chunk,
                        metadata={"source_py": doc.metadata["source"]}
                    ))
        except SyntaxError:
            # Если синтаксис некорректен, чанкировать как текст
            chunks = text_splitter.split_text(code)
            for chunk in chunks:
                py_chunks.append(Document(
                    page_content=chunk,
                    metadata={"source_py": doc.metadata["source"]}
                ))
    
    return py_chunks

# 4. Связывание чанков
def link_chunks(md_chunks, py_chunks, directory):
    """Связывает чанки .md и .py файлов через автоматически созданный словарь связей."""
    md_files = [chunk.metadata["source_md"] for chunk in md_chunks if "source_md" in chunk.metadata]
    file_relations = build_file_relations(md_files, directory)
    
    for md_chunk in md_chunks:
        md_file = os.path.basename(md_chunk.metadata.get("source_md", ""))
        if md_file in file_relations:
            md_chunk.metadata["related_files"] = file_relations[md_file]
    
    return md_chunks + py_chunks

# 5. Сохранение в векторную базу
def store_chunks(chunks):
    """Сохраняет чанки в векторной базе Chroma."""
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    vectorstore = Chroma.from_documents(chunks, embeddings)
    return vectorstore.as_retriever()

# 6. Тестирование ретривера
def test_retriever(retriever):
    """Тестирует ретривер на различных запросах."""
    test_queries = [
        "How to use UnstructuredMarkdownLoader?",
        "Show example of document loading in LangChain",
        "What is the purpose of RecursiveCharacterTextSplitter?",
        "Code example for Chroma vector store"
    ]
    
    for query in test_queries:
        print(f"\nTesting query: {query}")
        results = retriever.get_relevant_documents(query)
        for i, doc in enumerate(results[:2], 1):  # Ограничимся 2 результатами для краткости
            print(f"Result {i}:")
            print(f"Content: {doc.page_content[:200]}...")  # Первые 200 символов
            print(f"Metadata: {doc.metadata}\n")

In [20]:
API_KEY = "MGNlMTdlYzMtYjk3OS00ZmVlLTkzYzQtMGVmZmIyM2NkMDIzOjJlMWExNjM1LTIyZDAtNDdkMy04MmFhLTFkMDc2MDk5Y2ViMw=="

In [21]:
import os
import re
from typing import List
from langchain_community.document_loaders import UnstructuredMarkdownLoader, PythonLoader, DirectoryLoader
from langchain_text_splitters import MarkdownHeaderTextSplitter, RecursiveCharacterTextSplitter
from langchain_gigachat import GigaChatEmbeddings
from langchain.vectorstores import Chroma
from langchain.schema import Document
from langchain.vectorstores.faiss import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
import ast
from sentence_transformers import util
import numpy as np

# 1. Автоматизация связывания: Поиск упоминаний .py файлов в .md файлах
def extract_py_references(md_content):
    """Извлекает имена .py файлов из Markdown контента с помощью регулярных выражений."""
    py_files = set()
    pattern = r'(?:example|see|file|code)\s+([\w_]+\.py)'
    matches = re.findall(pattern, md_content, re.IGNORECASE)
    py_files.update(matches)
    pattern_fallback = r'[\w_]+\.py'
    matches_fallback = re.findall(pattern_fallback, md_content)
    py_files.update(matches_fallback)
    return list(py_files)

def build_file_relations(md_files, directory):
    """Создает словарь связей между .md и .py файлами."""
    file_relations = {}
    for md_file in md_files:
        with open(md_file, 'r', encoding='utf-8') as f:
            content = f.read()
        py_references = extract_py_references(content)
        existing_py_files = [
            os.path.join(root, f) for root, _, files in os.walk(directory) for f in files
            if f in py_references and f.endswith('.py')
        ]
        file_relations[os.path.basename(md_file)] = existing_py_files
    return file_relations

# # 2. Семантическое связывание с GigaChatEmbeddings
# def link_chunks_semantically(md_chunks, py_chunks, credentials, scope="GIGACHAT_API_PERS"):
#     """Связывает чанки .md и .py на основе семантического сходства."""
#     embeddings = GigaChatEmbeddings(credentials=credentials, scope=scope, verify_ssl_certs=False)
    
#     # Фильтрация и разбиение длинных чанков
#     max_tokens = 500  # Чуть меньше лимита GigaChat
#     text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)
    
#     filtered_md_chunks = []
#     for chunk in md_chunks:
#         if len(chunk.page_content) > max_tokens:
#             split_chunks = text_splitter.split_text(chunk.page_content)
#             for split_chunk in split_chunks:
#                 filtered_md_chunks.append(Document(
#                     page_content=split_chunk,
#                     metadata=chunk.metadata
#                 ))
#         else:
#             filtered_md_chunks.append(chunk)
    
#     filtered_py_chunks = []
#     for chunk in py_chunks:
#         if len(chunk.page_content) > max_tokens:
#             split_chunks = text_splitter.split_text(chunk.page_content)
#             for split_chunk in split_chunks:
#                 filtered_py_chunks.append(Document(
#                     page_content=split_chunk,
#                     metadata=chunk.metadata
#                 ))
#         else:
#             filtered_py_chunks.append(chunk)
    
#     # Получение эмбеддингов
#     md_texts = [chunk.page_content for chunk in filtered_md_chunks]
#     py_texts = [chunk.page_content for chunk in filtered_py_chunks]
#     md_embeddings = embeddings.embed_documents(md_texts)
#     py_embeddings = embeddings.embed_documents(py_texts)
    
#     similarities = util.cos_sim(md_embeddings, py_embeddings).numpy()
    
#     threshold = 0.75
#     for i, md_chunk in enumerate(filtered_md_chunks):
#         related_files = md_chunk.metadata.get("related_files", [])
#         for j, py_chunk in enumerate(filtered_py_chunks):
#             if similarities[i][j] > threshold:
#                 related_files.append(py_chunk.metadata["source_py"])
#         md_chunk.metadata["related_files"] = list(set(related_files))
    
#     return filtered_md_chunks + filtered_py_chunks

def link_chunks_semantically(md_chunks, py_chunks):
    """Связывает чанки .md и .py на основе семантического сходства."""
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    
    md_texts = [chunk.page_content for chunk in md_chunks]
    py_texts = [chunk.page_content for chunk in py_chunks]
    md_embeddings = embeddings.embed_documents(md_texts)
    py_embeddings = embeddings.embed_documents(py_texts)
    
    similarities = util.cos_sim(md_embeddings, py_embeddings).numpy()
    
    threshold = 0.75
    for i, md_chunk in enumerate(md_chunks):
        related_files = md_chunk.metadata.get("related_files", [])
        for j, py_chunk in enumerate(py_chunks):
            if similarities[i][j] > threshold:
                related_files.append(py_chunk.metadata["source_py"])
        md_chunk.metadata["related_files"] = list(set(related_files))
    
    return md_chunks + py_chunks

# 3. Загрузка и чанкирование .md файлов
def load_md_files(directory):
    """Загружает и чанкирует .md файлы с учетом структуры заголовков."""
    loader = DirectoryLoader(
        directory,
        glob="**/*.md",
        loader_cls=UnstructuredMarkdownLoader,
        loader_kwargs={"mode": "elements", "strategy": "fast"},
        silent_errors=True
    )
    docs = loader.load()
    
    headers_to_split_on = [("#", "Header 1"), ("##", "Header 2"), ("###", "Header 3")]
    splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)  # Уменьшен chunk_size
    
    md_chunks = []
    for doc in docs:
        if doc.metadata.get("category") == "NarrativeText":
            chunks = splitter.split_text(doc.page_content)
            for chunk in chunks:
                chunk.metadata.update({"source_md": doc.metadata["source"], "content_type": "docs"})
                md_chunks.append(chunk)
        else:
            chunks = text_splitter.split_text(doc.page_content)
            for chunk in chunks:
                md_chunks.append(Document(
                    page_content=chunk,
                    metadata={"source_md": doc.metadata["source"], "content_type": "docs"}
                ))
    
    return md_chunks

# 4. Загрузка и чанкирование .py файлов
def load_py_files(directory):
    """Загружает и чанкирует .py файлы с учетом структуры кода."""
    loader = DirectoryLoader(
        directory,
        glob="**/*.py",
        loader_cls=PythonLoader,
        silent_errors=True
    )
    docs = loader.load()
    
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
    py_chunks = []
    
    for doc in docs:
        code = doc.page_content
        try:
            tree = ast.parse(code)
            for node in ast.walk(tree):
                if isinstance(node, (ast.FunctionDef, ast.ClassDef)):
                    start_line = node.lineno
                    end_line = node.end_lineno
                    chunk = "\n".join(code.splitlines()[start_line-1:end_line])
                    py_chunks.append(Document(
                        page_content=chunk,
                        metadata={"source_py": doc.metadata["source"], "content_type": "code"}
                    ))
        except SyntaxError:
            chunks = text_splitter.split_text(code)
            for chunk in chunks:
                py_chunks.append(Document(
                    page_content=chunk,
                    metadata={"source_py": doc.metadata["source"], "content_type": "code"}
                ))
    
    return py_chunks

# 5. Связывание чанков
def link_chunks(md_chunks, py_chunks, directory, credentials = None, scope="GIGACHAT_API_PERS"):
    """Связывает чанки .md и .py файлов через регулярные выражения и семантическое сходство."""
    md_files = [chunk.metadata["source_md"] for chunk in md_chunks if "source_md" in chunk.metadata]
    file_relations = build_file_relations(md_files, directory)
    
    for md_chunk in md_chunks:
        md_file = os.path.basename(md_chunk.metadata.get("source_md", ""))
        if md_file in file_relations:
            md_chunk.metadata["related_files"] = file_relations[md_file]
    
    all_chunks = link_chunks_semantically(md_chunks, py_chunks)
    return all_chunks

# 6. Сохранение в векторную базу
def store_chunks(chunks: List[Document]):
    print(f"Processing {len(chunks)} chunks")
    if not chunks:
        raise ValueError("Список чанков пуст, невозможно создать векторную базу")
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    vector_store = FAISS.from_documents(chunks, embeddings)
    print("Создана новая векторная база FAISS")
    return vector_store.as_retriever()

# 7. Тестирование ретривера с учетом предпочтений пользователя
def test_retriever(retriever, queries=None):
    """Тестирует ретривер на различных запросах с учетом типа контента."""
    if queries is None:
        queries = [
            "Show a code example of creating a ReAct agent using LangGraph's create_react_agent function.",  # Предпочтение: документация
            "Provide a code example of using the ainvoke method to asynchronously invoke a LangChain Runnable.",  # Предпочтение: код
            "Show a code example of using SelfQueryRetriever to convert a natural language query into metadata filters.",  # Предпочтение: документация
            "What is the purpose of the langchain-core package in the LangChain framework?" 
            "What is the difference between older string-in, string-out LLMs and newer Chat Models in LangChain?"
        ]
    
    for query in queries:
        print(f"\nTesting query: {query}")
        # Определяем предпочтение: код или документация
        prefer_code = any(keyword in query.lower() for keyword in ["code", "example", "script"])
        content_type = "code" if prefer_code else "docs"
        
        results = retriever.get_relevant_documents(query)
        # Фильтрация результатов по типу контента
        filtered_results = [
            doc for doc in results if doc.metadata.get("content_type") == content_type
        ] or results[:2]  # Возвращаем все, если фильтр не дал результатов
        
        for i, doc in enumerate(filtered_results[:2], 1):
            print(f"Result {i}:")
            print(f"Content: {doc.page_content}")
            print(f"Metadata: {doc.metadata}\n")

In [22]:
def main(directory, credentials, scope="GIGACHAT_API_PERS"):
    """Основной процесс обработки документации."""
    md_chunks = load_md_files(f"{directory}/md")
    py_chunks = load_py_files(f"{directory}/py")
    all_chunks = link_chunks(md_chunks, py_chunks, directory, credentials, scope)
    retriever = store_chunks(all_chunks)
    test_retriever(retriever)
    return retriever

In [23]:
directory = "test_directory" 
credentials = API_KEY
scope = "GIGACHAT_API_PERS"
retriever = main(directory, credentials, scope)

Processing 298 chunks
Создана новая векторная база FAISS

Testing query: Show a code example of creating a ReAct agent using LangGraph's create_react_agent function.
Result 1:
Content: def create_react_agent(
    llm: BaseLanguageModel,
    tools: Sequence[BaseTool],
    prompt: BasePromptTemplate,
    output_parser: Optional[AgentOutputParser] = None,
    tools_renderer: ToolsRenderer = render_text_description,
    *,
    stop_sequence: Union[bool, list[str]] = True,
) -> Runnable:
    """Create an agent that uses ReAct prompting.

    Based on paper "ReAct: Synergizing Reasoning and Acting in Language Models"
    (https://arxiv.org/abs/2210.03629)

    .. warning::
       This implementation is based on the foundational ReAct paper but is older and not well-suited for production applications.
       For a more robust and feature-rich implementation, we recommend using the `create_react_agent` function from the LangGraph library.
       See the [reference doc](https://langchain-ai.git